# Preparation
Preparation before main execution.

## Preferences

In [ ]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Set working directory
full_path = '/content/drive/MyDrive/Periaortic_fai/'

In [ ]:
# Import library
# Basic
import os
import glob
from tqdm.notebook import tqdm
import re
import random
import shutil
import datetime

# Data process
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import cv2

# CT process
!pip install pylibjpeg-libjpeg
!pip install pylibjpeg
!pip install pydicom
import pydicom

## Install Detectron2

In [ ]:
# pip install
!python -m pip install pyyaml --upgrade
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
# Show versions
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch:", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

In [ ]:
# Import Detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode

# Define functions
Define functions to be used for processing.

In [ ]:
# Sort functions
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [ ]:
# Main
def run(file_path, debug=False):
    ## Load
    # Load DICOM file
    dicom = pydicom.dcmread(file_path)
    file_name = os.path.basename(file_path)

    # Get necessary values from DICOM daa
    dp = dicom.pixel_array.copy()  # CT value
    intercept = dicom['RescaleIntercept'].value  # Intercept
    try:
        outer_value = dicom['PixelPaddingValue'].value  # CT value outside the circle
    except:
        outer_value = -2048  # Default Value

    # Inverse transform
    dp = dp + intercept
    outer_value += intercept
    dp = np.where(dp==outer_value, -2048, dp)

    # DICOM -> Image
    if debug:
        save_path = f"{file_name}.jpg"
    else:
        save_path = f"{org_path}{file_name}.jpg"
    cv2.imwrite(save_path, dp)

    # Load the image
    im = cv2.imread(save_path)
    h, w = im.shape[0], im.shape[1]
    if debug:
        plt.figure(figsize=(20,20))
        plt.subplot(2,2,1)
        plt.imshow(im)

    ## Predict
    # Predict image
    outputs = predictor(im)

    # Save results
    v = Visualizer(im[:, :, ::-1], scale=1.0)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    if debug:
        plt.subplot(2,2,2)
        plt.imshow(v.get_image()[:, :, ::-1])
    else:
        cv2.imwrite(f"{pred_path}{file_name}.jpg", v.get_image()[:, :, ::-1])

    ## Postprocess
    # Prediction results
    pred_masks = outputs['instances'].get('pred_masks')
    if len(pred_masks) == 0:
        if debug:
            print(f"[{file_name}] Theres no prediction result.")
        return [{'result': file_name}]  # Exit if no prediction result exists

    result_list = []
    for i, pred_mask in enumerate(pred_masks):  # Loop as many times as the prediction results
        # Get scores and classes
        pred_mask = pred_mask.cpu().numpy().astype(np.uint8)
        score = outputs['instances'].get('scores').cpu().numpy()[i]
        pred_classe = outputs['instances'].get('pred_classes').cpu().numpy()[i]

        # Get center coordinates
        index = np.where(pred_mask)
        x_mean = round(np.sum(index[0]) / len(index[0]), 2)
        y_mean = round(np.sum(index[1]) / len(index[1]), 2)

        # Check inside/outside detection range
        if (x_mean < x_range[0]) | (x_range[1] < x_mean) | (y_mean < y_range[0]) | (y_range[1] < y_mean) :
            # Exit if out of range
            print(f"[{file_name}][{i+1}] Out of range: ({x_mean}, {y_mean})")
            continue
        else:
            # Rename and continue if in range
            if i == 0:
                file_name_tmp = file_name
            else:
                file_name_tmp = f"{file_name}_{i+1}"

        ## Extraction of outer region
        # PixelSpacing conversion
        ps_value = dicom['PixelSpacing'].value
        if debug:
            print(f"[{file_name_tmp}] Pixel Spacing: {ps_value}")
        if ps_value[0] != ps_value[1]:
            print(f"[Warning] Check the value of Pixel Spacing: {ps_value}")
        radius_trans = int(radius / ps_value[0])

        # Draw a circle centered on the predicted result pixel
        cir = np.zeros((h, w, 3))
        for hight in range(0, h):
            for width in range(0, w):
                if pred_mask[width][hight] == 1:  # Predicted pixel
                    # Draw on zeros image
                    cv2.circle(cir,
                              center=(hight, width),
                              radius=radius_trans,
                              color=(0, 255, 0),
                              thickness=-1,
                              lineType=cv2.LINE_4,
                              shift=0)

        # Draw only the mask of the outer doughnut-shaped area by subtracting the predicted results
        cir_mask = (cir[:,:,1]).astype(np.uint8)  # Extract only green and convert to uint8
        target_mask = np.where((pred_mask==0)&(cir_mask!=0), 1, 0)  # Extract outer region
        if debug:
            plt.subplot(2,2,3)
            plt.imshow(target_mask)

        # Draw a mask of the outer area on the original image
        cir_on_img = im.copy()
        cir_on_img[:,:,1] = np.where(target_mask == 1, 255, cir_on_img[:,:,1])  # Extract outer region
        if debug:
            plt.imshow(cir_on_img)
        else:
            cv2.imwrite(f"{mask_path}{file_name_tmp}.jpg", cir_on_img)

        ## Extract target region
        # Leave only the CT values of target region
        target_CT = dp * target_mask
        if debug:
            print(f"[{file_name_tmp}][{select_category[pred_classe]}] Before selection…　Min: {target_CT.min()}  Max: {target_CT.max()}  Area: {target_mask.sum()}")

        # Select target CT value
        target_CT = np.where(target_CT > max_CT, 0, target_CT)  # Converts more than the maximum value to 0
        target_CT = np.where(target_CT < min_CT, 0, target_CT)  # Converts less than minimum value to 0
        if debug:
            print(f"[{file_name_tmp}][{select_category[pred_classe]}] After selection…　Min: {target_CT.min()}  Max: {target_CT.max()}  Area: {target_mask.sum()}")

        # Extract only the CT values contained in target_CT
        CTs = []
        for hight in range(0, h):
            for width in range(0, w):
                if target_mask[width][hight] == 1:  # Predicted pixel
                    if target_CT[width][hight] != 0:  # in target_CT
                        CTs.append(target_CT[width][hight])

        # Draw Target CT mask
        calc_on_img = im.copy()
        calc_on_img[:,:,1] = np.where(target_CT != 0, 255, calc_on_img[:,:,1])
        if debug:
            print(f"[{file_name_tmp}][{select_category[pred_classe]}] Statistic…  Mean: {np.mean(CTs)}  Standard deviation: {np.std(CTs)}  Area: {len(CTs)}  Total: {np.sum(CTs)}")
            plt.subplot(2,2,4)
            plt.imshow(calc_on_img)
            os.makedirs(f"{full_path}output/debug/", exist_ok=True)
            plt.savefig(f"{full_path}output/debug/{file_name_tmp}.jpg")
        else:
            cv2.imwrite(f"{res_path}{file_name_tmp}.jpg", calc_on_img)

        # Prepare results dict
        _dict = {}
        _dict['result'] = file_name_tmp
        _dict['class'] = select_category[pred_classe]
        _dict['CTs'] = CTs
        _dict['score'] = score
        _dict['(x,y)'] = (x_mean, y_mean)

        # Add dict to results list
        result_list.append(_dict)

    return result_list

# Main
- Predict the aortic region by a trained model.<br>
- Calculate CT values around the aortic region.

## Settings

In [ ]:
# input / output path
input_root = f"{full_path}input/"
output_root = f"{full_path}output/"

# CT value detection settings
threshold = 0.90      # confidence threshold
x_range = [150, 400]  # target range: x-axis
y_range = [200, 400]  # target range: y-axis
radius = 5            # distance around the aortic region (radius in mm)
min_CT = -190         # Minimum CT value
max_CT = -30          # Maximum CT value

In [ ]:
# About trained model
suffix = "20240513"
# suffix = "YYYYMMDD"
trained_dir = "COCO-InstanceSegmentation"
trained_yaml = "mask_rcnn_X_101_32x8d_FPN_3x"
checkpoint = "model_final.pth"
select_category = ['Ascending aorta', 'Descending aorta']

## Load

In [ ]:
# Load trained model
trained_model = f"{trained_dir}/{trained_yaml}.yaml"
weights_dir = f"{full_path}weights/{trained_yaml}_{suffix}"

# Create predictor instance
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(trained_model))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = threshold
cfg.MODEL.WEIGHTS = os.path.join(weights_dir, checkpoint)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(select_category)
predictor = DefaultPredictor(cfg)

In [ ]:
# Get all folders in input folder
files = os.listdir(input_root)
dirs = []
for f in files:
    path = os.path.join(input_root, f)
    if os.path.isdir(path):
        dirs.append(f)
print(f"Target folder names：{dirs}")

## Test execution
Randomly executes only one file.

In [ ]:
# Get a list of files in the first folder
input_path = f"{input_root}/{dirs[0]}/"
output_path = f"{output_root}/{dirs[0]}/"
files_path = glob.glob(f"{input_path}*")
files_path = sorted(files_path, key=natural_keys)

# Choice one for test
test_file = random.choice(files_path)
print(f"Test target: {test_file}")

# Executes for test
list_results = run(test_file, debug=True)

## Main execution

In [ ]:
# Loop all folders
for target in dirs:
    # Set path
    print(f"[{target}] Start: {datetime.datetime.now()}")
    input_path = f"{input_root}/{target}/"
    output_path = f"{output_root}/{target}/"

    # Get a list of files
    files_path = glob.glob(f"{input_path}*")
    files_path = sorted(files_path, key=natural_keys)

    # Delete output folder if it already exists
    if os.path.exists(output_path):
        print(f'Remove existed output folder...')
        shutil.rmtree(output_path)

    # Make folders
    org_path = f"{output_path}original_images/"
    pred_path = f"{output_path}predicted_images/"
    mask_path = f"{output_path}masked_images/"
    res_path = f"{output_path}result_images/"
    os.makedirs(org_path, exist_ok=True)
    os.makedirs(pred_path, exist_ok=True)
    os.makedirs(mask_path, exist_ok=True)
    os.makedirs(res_path, exist_ok=True)

    # Loop all files
    _list = []
    for file_path in tqdm(files_path):
        # Execute
        list_results = run(file_path, debug=False)

        for dict_result in list_results:
            # Store CT value data
            try:
                CT = dict_result['CTs']

                dict_result["mean"] = np.mean(CT)
                dict_result["std"] = np.std(CT)
                dict_result["area"] = len(CT)
                dict_result["sum"] = np.sum(CT)
                _list.append(dict_result)

            except Exception as e:
                print(f"[{dict_result['result']}] Theres no prediction result.")

    # Output CT value data
    df = pd.DataFrame(_list)
    df.to_csv(f"{output_path}{target}.csv", index=None)
    print(f"[{target}] End: {datetime.datetime.now()}\n")